In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fashion_mnist=keras.datasets.fashion_mnist

In [3]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [4]:
train_images=train_images/255.0
test_images=test_images/255.0

In [5]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [6]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [9]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [10]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [11]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 07m 19s]
val_accuracy: 0.9068333506584167

Best val_accuracy So Far: 0.909166693687439
Total elapsed time: 00h 44m 42s


In [12]:
model=tuner_search.get_best_models(num_models=1)[0]

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 32)        25632     
                                                                 
 flatten (Flatten)           (None, 12800)             0         
                                                                 
 dense (Dense)               (None, 96)                1228896   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 1256330 (4.79 MB)
Trainable params: 1256330 (4.79 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 111s 65ms/step - loss: 0.1685 - accuracy: 0.9376 - val_loss: 0.2705 - val_accuracy: 0.9087
Epoch 5/10
1688/1688 [==============================] - 106s 63ms/step - loss: 0.1325 - accuracy: 0.9501 - val_loss: 0.2603 - val_accuracy: 0.9163
Epoch 6/10
1688/1688 [==============================] - 103s 61ms/step - loss: 0.1008 - accuracy: 0.9624 - val_loss: 0.2799 - val_accuracy: 0.9117
Epoch 7/10
1688/1688 [==============================] - 752s 446ms/step - loss: 0.0754 - accuracy: 0.9721 - val_loss: 0.3172 - val_accuracy: 0.9117
Epoch 8/10
1688/1688 [==============================] - 106s 63ms/step - loss: 0.0604 - accuracy: 0.9776 - val_loss: 0.3607 - val_accuracy: 0.9105
Epoch 9/10
1688/1688 [==============================] - 110s 65ms/step - loss: 0.0512 - accuracy: 0.9811 - val_loss: 0.3685 - val_accuracy: 0.9118
Epoch 10/10
1688/1688 [==============================] - 105s 62ms/step - loss: 0.0375 - accuracy: 0.9865 - val_loss: